# Visualisation

In [7]:
import folium, pandas

base_data_url = "../datasets/project/road safety/road-safety-data-{}"
accidents_16 = pandas.read_csv(base_data_url.format("accidents-2016.csv"))
accidents_15 = pandas.read_csv(base_data_url.format("accidents-2015.csv"))
accidents_14 = pandas.read_csv(base_data_url.format("accidents-2005-2014.csv"))
accidents = pandas.DataFrame()
accidents = pandas.concat([accidents_16, accidents_15, accidents_14], axis=0)

In [8]:
accidents = pandas.DataFrame()
accidents = pandas.concat([accidents_16, accidents_15, accidents_14], axis=0)

accidents_n = accidents.loc[(accidents["Latitude"] != "Data missing or out of range") &
             (accidents["Longitude"] != "Data missing or out of range")]
# Convert lat/long fields to numeric, or we have a crash
accidents_n[['Longitude','Latitude']] = accidents_n[['Longitude','Latitude']].apply(pandas.to_numeric)

/usr/lib64/python3.6/site-packages/pandas/core/frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


## Accidents map
Show the location of all accidents, color coded by severity.

In [9]:
map_acc = folium.Map(location=[56.4620, -2.9707], zoom_start=12)
colors = {'Slight': 'green', 'Serious': 'orange', 'Fatal': 'red'}
accidents_n.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                               radius=10, 
                                               fill_color=colors[row["Accident_Severity"]],
                                               fill = True,
                                               stroke=False,
                                               fill_opacity=0.5,
                                               popup = str(row["Accident_Index"]))
                                             .add_to(map_acc), axis=1)

map_acc.save("map_acc.html")

## Cameras
Show the locations of all cameras in Dundee, and Voronoi polygons for them. Allows you to determine the nearest camera to you at a given location.

In [10]:
import numpy
cameras = pandas.read_csv("../datasets/project/cameras_wgs.csv", delimiter='\t')
# Need to filter out nans here
cameras = cameras[numpy.isfinite(cameras['Lat'])]

In [11]:
cameras_coords = pandas.concat([cameras.Lat, cameras.Lon], axis=1)

In [12]:
def notnan(x):
    try: 
        int(x)
        return True
    except:
        return False

In [13]:
from scipy.spatial import Voronoi, voronoi_plot_2d
cameras_list = [list(x) for x in cameras_coords.values if notnan(x[0])]

vor = Voronoi(cameras_list) 

In [14]:
%%capture
# Now actually plot with folium
map_cam = folium.Map(location=[56.4620, -2.9707], zoom_start=12)

cameras.apply(lambda row:folium.Circle(location=[row["Lat"], row["Lon"]], 
                                       radius=1, 
                                       color='black',
                                       popup = row["Camera Location"])
                                      .add_to(map_cam), axis=1)

In [15]:
# Add voronois
# http://comet.lehman.cuny.edu/owen/teaching/datasci/makeVor.py

from geojson import FeatureCollection, Feature, Polygon

vorJSON = open('libVor.json', 'w')
point_voronoi_list = []
feature_list = []
for region in range(len(vor.regions)-1):
    vertex_list = []
    for x in vor.regions[region]:
        #Not sure how to map the "infinite" point, so, leave off those regions for now:
        if x == -1:
            break;
        else:
            #Get the vertex out of the list, and flip the order for folium:
            vertex = vor.vertices[x]
            vertex = (vertex[1], vertex[0])
        vertex_list.append(vertex)
    #Save the vertex list as a polygon and then add to the feature_list:
    polygon = Polygon([vertex_list])
    feature = Feature(geometry=polygon, properties={})
    feature_list.append(feature)

#Write the features to the new file:
feature_collection = FeatureCollection(feature_list)
print (feature_collection, file=vorJSON)
vorJSON.close()


folium.GeoJson(
    "libVor.json",
    name='geojson'
).add_to(map_cam)
map_cam.save("map_cam.html")

In [16]:
#map_cam

# Other stuff

## Food standards
Show the location of restaurants in Dundee, and their color-coded food safety rating.

In [17]:
food = pandas.read_csv("../datasets/fhrs.csv")
# Filter nans
food = food[numpy.isfinite(food["EstablishmentDetail - Geocode - Latitude"])]

# Remove duplicates
food.drop_duplicates(subset=['FHRSID'], keep='first')

# Make dataset smaller
#food = food.iloc[:100]
print()

In [18]:
set(food["EstablishmentDetail - RatingValue"].values)

{'Awaiting Inspection', 'Exempt', 'Improvement Required', 'Pass'}

In [19]:
import re
map_food = folium.Map(location=[56.4620, -2.9707], zoom_start=12)
colors = {'Awaiting Inspection': 'amber', 'Exempt': 'blue', 'Improvement Required': 'red', 'Pass': 'green'}

def label(row):
    label = ""
    label += (re.sub(r'\W+', ' ', str(row["EstablishmentDetail - BusinessName"]))) + " - "
    label += row["EstablishmentDetail - RatingValue"]
    return label

food.apply(lambda row:folium.CircleMarker(location=[row["EstablishmentDetail - Geocode - Latitude"], 
                                                    row["EstablishmentDetail - Geocode - Longitude"]],
                                         radius = 10,
                                         stroke = False,
                                         fill = True,
                                         fill_color = colors[row["EstablishmentDetail - RatingValue"]],
                                         fill_opacity = 0.5,
                                          # Regex is necessary, otherwise crash
                                          popup = label(row))
                                         .add_to(map_food), axis=1)
map_food.save("map_food.html")

## Monuments
- Which monuments am I in walking distance of?

In [127]:
monuments = pandas.read_csv("../datasets/monuments_transformed.csv")

In [131]:
from folium.plugins import MeasureControl

map_monuments = folium.Map(location=[56.4620, -2.9707], zoom_start=12)
# Monuments points
monuments.apply(lambda row:folium.Marker(location=[row["Latitude"], row["Longitude"]], 
                                                      #icon=folium.Icon(color=color(row), icon=""),
                                       popup = re.sub(r'\W+', ' ',str(row["Monument_Name"])))
                                      .add_to(map_monuments), axis=1)
train_station_coords = [56.4568, -2.9702]


map_monuments.add_child(MeasureControl())